In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from collections import defaultdict
import random
import sklearn.metrics

In [256]:
num_nodes = 23259
num_feats = 300
num_labels = 4
feat_data, ori_adj_lists, pos_triplets = load_dblp()
features = nn.Embedding(num_nodes, num_feats)
features.weight = nn.Parameter(torch.FloatTensor(feat_data), requires_grad=False)

In [257]:
import copy
adj_lists = copy.deepcopy(ori_adj_lists)
for i in test_idx:
    if labels[i]==1:
        sub, rel, obj = triplets[i]
        adj_lists[rel][sub].remove(obj)

In [20]:
[len(np.where(pos_triplets[:,1]==rel_idx)[0]) for rel_idx in range(10)]

[333370, 53544, 129246, 129246, 11413, 11413, 11701, 11701, 25735, 25735]

In [43]:
def rel_sampling(pos_triplets):
    # according to the number of relations to choose a relation
    rel_cnt = np.array([len(np.where(pos_triplets[:,1]==
                           rel_idx)[0]) for rel_idx in range(10)])
    rel_prob = rel_cnt / np.sum(rel_cnt)
    return np.random.choice(np.arange(10), p=rel_prob)

In [41]:
def rel_triplets_smapling(rel_idx, triplets, train_idx):
    # sample triplets of batch 256 in this relation
    rel_train_idx = train_idx[triplets[train_idx,1]==rel_idx]
    return np.array(rel_train_idx[:256])

In [59]:
num_rel = 10
def cnt_node_rel(triplets):
    node_rel_table = np.zeros((num_nodes, num_rel))
    for i in range(num_nodes):
        for j in range(num_rel):
            node_rel_table[i,j] = len(np.intersect1d(
                np.where(triplets[:,0]==i),np.where(triplets[:,1]==j)))
    return node_rel_table

In [63]:
cnt_node_rel(pos_triplets)

KeyboardInterrupt: 

In [55]:
np.intersect1d(np.where(triplets[:,0]==1),np.where(triplets[:,1]==0))

array([      1,      12,      14, ..., 8121447, 8121467, 8121471])

In [58]:
triplets[12], len(np.intersect1d(np.where(triplets[:,0]==1),np.where(triplets[:,1]==0)))

(array([1, 0, 2]), 2921)

In [35]:
triplets[train_idx,1]==0, train_idx[triplets[train_idx,1]==0]

(array([ True, False, False, ...,  True,  True,  True]),
 array([ 491351, 4606956, 2151503, ..., 4724836, 4493213, 6448727]))

In [39]:
len(train_idx)

6539312

In [38]:
len(train_idx[triplets[train_idx,1]==0]),len(triplets[train_idx,1]==0)

(2932948, 6539312)

In [31]:
rel_sampling(pos_triplets)

0

In [8]:
len(train_idx), len(test_idx)

(6539312, 1634832)

In [5]:
def negative_sampling(triplets, num_nodes):
#     print(len(triplets))
    labels = np.ones((len(triplets)*(10+1),1))
    labels[len(triplets):] = 0
    # random.shuffle(labels)
    # triplets = np.array(triplets)
    neg_num = 10 * len(triplets)
    neg_triplets = np.tile(triplets, (10, 1))
    neg_nodes = np.random.randint(num_nodes, size=neg_num)
    neg_choice = np.random.uniform(size=neg_num)
    neg_sub_idx, neg_obj_idx = neg_choice>0.5, neg_choice<=0.5
    neg_triplets[neg_sub_idx,0] = neg_nodes[neg_sub_idx] 
    neg_triplets[neg_obj_idx,2] = neg_nodes[neg_obj_idx]
    return np.concatenate((triplets, neg_triplets),axis=0), labels

In [6]:
triplets, labels = negative_sampling(pos_triplets, num_nodes)

In [7]:
random_idx = np.random.permutation(len(triplets))
train_idx = random_idx[:len(triplets)//10*8]
test_idx = random_idx[len(triplets)//10*8:]
len(train_idx)

6539312

In [42]:
for p in list(rgcn_graphsage.parameters()):
    print(p.shape)

torch.Size([10, 128])
torch.Size([10, 300, 128])
torch.Size([10, 128, 128])
torch.Size([23259, 300])


In [259]:
for i in range(50):
    batch_test_output = rgcn_graphsage.forward(triplets[test_idx[i*256:(i+1)*256]])
    if i == 0:
        test_output = batch_test_output
    else:
        test_output = torch.cat((test_output, batch_test_output))
test_output[test_output>0.5]=1
test_output[test_output<=0.5]=0
print("Test Accuracy:", sklearn.metrics.accuracy_score(labels[test_idx[:50*256]],
                                                              test_output.detach().numpy()))

Test Accuracy: 0.913046875


In [95]:
#embedding
rgcn_graphsage.encoder.forward([1])

torch.Size([200, 344]) torch.Size([344, 300])
torch.Size([10, 20]) torch.Size([200, 128])


tensor([[0.0000, 0.0067, 0.0000, 0.0000, 0.0190, 0.0110, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0244, 0.0000, 0.0000, 0.0000, 0.0092, 0.0000, 0.0000, 0.0000,
         0.0228, 0.0000, 0.0136, 0.0052, 0.0043, 0.0000, 0.0000, 0.0107, 0.0000,
         0.0137, 0.0000, 0.0091, 0.0000, 0.0000, 0.0000, 0.0100, 0.0000, 0.0043,
         0.0136, 0.0000, 0.0000, 0.0044, 0.0100, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0132, 0.0057, 0.0000, 0.0188, 0.0000, 0.0000, 0.0004, 0.0186, 0.0195,
         0.0029, 0.0035, 0.0254, 0.0000, 0.0194, 0.0122, 0.0057, 0.0043, 0.0129,
         0.0126, 0.0231, 0.0000, 0.0111, 0.0000, 0.0000, 0.0000, 0.0138, 0.0171,
         0.0207, 0.0000, 0.0108, 0.0000, 0.0046, 0.0032, 0.0000, 0.0000, 0.0099,
         0.0167, 0.0080, 0.0246, 0.0000, 0.0000, 0.0223, 0.0221, 0.0100, 0.0000,
         0.0068, 0.0019, 0.0042, 0.0000, 0.0188, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0054, 0.0070, 0.0190, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0005, 0.0

In [11]:
type(features)

torch.nn.modules.sparse.Embedding

In [41]:

rgcn_graphsage.loss(batch_triplets, batch_labels)

torch.Size([10350, 6905]) torch.Size([6905, 300])
torch.Size([1000, 1035]) torch.Size([10350, 128])
torch.Size([10700, 6987]) torch.Size([6987, 300])
torch.Size([1000, 1070]) torch.Size([10700, 128])


tensor(0.6935, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [245]:
class Encoder(nn.Module):
    def __init__(self, features, num_feats, h_dim, num_rel, adj_lists):
        super(Encoder, self).__init__()
        self.aggregator_1 = Aggregator(adj_lists, num_feats)
#         self.feat_dim = features.shape[1]
        self.features = features
        self.weight_1 = nn.Parameter(
            torch.FloatTensor(num_rel, num_feats, h_dim))
        nn.init.xavier_uniform_(self.weight_1)
        
        self.aggregator_2 = Aggregator(adj_lists, h_dim, second_cut=True)
        self.weight_2 = nn.Parameter(
            torch.FloatTensor(num_rel, h_dim, h_dim))
        nn.init.xavier_uniform_(self.weight_2)
        
    def encode(self, nodes):
        aggr_feats, rel_idxs = self.aggregator_1.forward(nodes, self.features)
        x = F.relu(torch.bmm(aggr_feats, torch.index_select(self.weight_1, dim=0, index=rel_idxs)))
#         print('first cut x mean shape', x.mean(0).shape)
        return x.mean(0)
        
    def forward(self, nodes):
#         aggre_feat = self.aggregator_1.forward(nodes, self.features)
#         print(aggre_feat.type(),aggre_feat)
#         x = F.relu(torch.mm(self.aggregator_1.forward(nodes, self.features)
#                             , self.weight_1))
        aggr_feats, rel_idxs = self.aggregator_2.forward(nodes, self.encode)   
        x = F.relu(torch.bmm(aggr_feats, torch.index_select(self.weight_2, dim=0, index=rel_idxs)))
#         print('torch.index_select(self.weight_2, dim=0, rel_idx)', 
#               (torch.index_select(self.weight_2, dim=0, index=rel_idxs)).shape)
#         print('x.shape', x.shape)
        x = x.mean(0)
#         print('encode mean 0 x.shape', x.shape)
        return x 

In [119]:
rel_neighs = [[adj_lists[rel_idx][int(node)] for node in [0,1,2,3,4,5]]\
                        for rel_idx in [0,1,2,3,4,5,6,7,8]]

In [241]:
rgcn_graphsage.encoder.weight_2.requires_grad, rgcn_graphsage.weight_relation.requires_grad

(True, True)

In [258]:
rgcn_graphsage = RGCN_GraphSAGE(features, num_feats, 128, 10, adj_lists)
optimizer = torch.optim.Adam(rgcn_graphsage.parameters(), lr=0.3)
for i in range(30):
#     rel_idx = rel_sampling(pos_triplets)
    rel_idx = np.random.randint(10)
    bacth_train_idx = rel_triplets_smapling(rel_idx, triplets, train_idx)
#     bacth_train_idx = np.array(train_idx[:256])
    random.shuffle(train_idx)
    
    batch_triplets = triplets[bacth_train_idx]
    batch_labels = torch.FloatTensor(labels[bacth_train_idx])
    
    optimizer.zero_grad()
    loss = rgcn_graphsage.loss(batch_triplets, batch_labels)
    loss.backward()
    optimizer.step()
    print(i, rel_idx, loss)

0 3 tensor(0.6931, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
1 5 tensor(0.6933, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
2 4 tensor(2.9336, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
3 4 tensor(1.7260, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
4 1 tensor(0.6951, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
5 4 tensor(0.6919, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
6 5 tensor(0.6934, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
7 8 tensor(0.6869, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
8 3 tensor(0.6363, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
9 8 tensor(4.5786, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
10 8 tensor(0.8641, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
11 0 tensor(0.6024, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
12 5 tensor(1.0719, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
13 4 tensor(0.6966, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
14 1 tensor(0.6876, grad_fn=<BinaryCrossEntropyWithLogitsB

In [208]:
np.argsort(np.array([0.02,0.3,0.1]))[::-1]

array([1, 2, 0])

In [243]:
class Aggregator(nn.Module):
    def __init__(self, adj_lists, dim, second_cut=False):
        super(Aggregator, self).__init__()
#         self.features = features
        self.second_cut = second_cut
        self.adj_lists = adj_lists
        self.dim = dim
        
    def forward(self, nodes, features, num_sample=10):
        """
        nodes --- list of nodes in a batch
        to_neighs --- list of sets, each set is the set of neighbors for node in batch
        num_sample --- number of neighbors to sample. No sampling if None.
        """
        self.features=features
        
        # outgoing
#         print('nodes',nodes)
#         print('nodes[-1]',nodes[-1])
#         for adj in self.adj_lists:
#             print(len(adj[nodes[-1]]))
        nodes_rels_cnt = np.array([[len(adj_list[node]) for node in nodes]\
                  for adj_list in self.adj_lists])
#         print('print(nodes_rels_cnt)', nodes_rels_cnt)
        nodes_rels_cnt = nodes_rels_cnt.sum(1)
#         print('nodes_rels_cnt', nodes_rels_cnt)
#         print('np.sum(nodes_rels_cnt)', np.sum(nodes_rels_cnt))
        rel_prob = nodes_rels_cnt / np.sum(nodes_rels_cnt)
        # sample 3 relations
#         print('rel_prob',rel_prob)
        possible_num_rels = np.sum(rel_prob>=0.1)
        sort_rels_idxs = np.argsort(rel_prob)[::-1].copy()
        rel_idxs = sort_rels_idxs[:possible_num_rels]
#         rel_idxs = np.random.choice(10, p=rel_prob, size=3)
#         print('rel_idxs',rel_idxs)
        all_rels_neighs = [[self.adj_lists[rel_idx][node] for node in nodes]\
                        for rel_idx in rel_idxs]
        
        # totally sample 10 neighbors, came from 3 relations
#         rel_sample_neighs = []
        aggr_rel_feats = torch.Tensor(len(rel_idxs), len(nodes), self.dim)
        for r, rel_idx in enumerate(rel_idxs):
            rel_neighs = all_rels_neighs[r]
#             print('len(rel_neighs)',len(rel_neighs))
#             print('len(rel_prob)', len(rel_prob), rel_prob)
#             print('rel_idx',rel_idx)
            
            rel_sample_neighs = [set(random.sample(rel_neighs[i], int(10*rel_prob[rel_idx])))\
                                 if len(rel_neighs[i]) >= int(10*rel_prob[rel_idx]) else rel_neighs[i]\
                                 for i in range(len(nodes))]
#             print('rel_sample_neighs',rel_sample_neighs)
#             print('rel_sample_neighs',rel_sample_neighs)
            rel_sample_neighs_list = list(set.union(*rel_sample_neighs))
            # n: actual node index
            rel_sample_neighs_dict = {neigh:i for i,neigh in enumerate(rel_sample_neighs_list)}
            mask = torch.zeros(len(nodes), len(rel_sample_neighs_list))
            
            row_idxs = [i for i in range(len(nodes)) for _ in rel_sample_neighs[i]]
            col_idxs = [rel_sample_neighs_dict[neigh] for node_rel_sample_neighs in rel_sample_neighs\
                       for neigh in node_rel_sample_neighs]
#             print('row_idxs',row_idxs)
#             print('col_idxs',col_idxs)
            
            mask[row_idxs, col_idxs] = 1
#             print('len(rel_sample_neighs_list)', len(rel_sample_neighs_list))
            
            # to deal with np.nan
            num_neigh = mask.sum(1, keepdim=True)
            thre_zeros = torch.zeros_like(mask)
            # num_neigh = mask.sum()/(mask.shape[0])
            # mask = mask.div(num_neigh)
            mask = torch.where(num_neigh != 0, mask.div(num_neigh), thre_zeros)
            
            if self.second_cut:
                embed_matrix = self.features(rel_sample_neighs_list)
            else:
                embed_matrix = self.features(torch.LongTensor(rel_sample_neighs_list))
            
#             print('mask.shape, embed_matrix.shape',mask.shape, embed_matrix.shape)
            aggr_rel_feats[r,:,:] = torch.mm(mask, embed_matrix)
#             print('aggr_rel_feats.requires_grad',aggr_rel_feats.requires_grad)
#             print('embed_matrix.requires_grad',embed_matrix.requires_grad)
        
#         for i in len(rel_idxs):
#             mask = torch.zeros(len(nodes), len(unique_nodes))
#         multi_to_neighs = [[single_adj_lists[int(node)] for node in nodes]\
#                      for single_adj_lists in self.adj_lists]
#         _set = set
#         if not num_sample is None:
#             _sample = random.sample
#             multi_samp_neighs = [[_set(_sample(to_neigh, 
#                             num_sample,
#                             )) if len(to_neigh) >= num_sample else to_neigh for to_neigh in to_neighs]
#                             for to_neighs in multi_to_neighs]
#         else:
#             multi_samp_neighs = to_neighs

        # print(multi_samp_neighs)

#         if self.gcn:
#             samp_neighs = [samp_neigh + set([nodes[i]]) for i, samp_neigh in enumerate(samp_neighs)]
#         multi_unique_nodes_list = [set.union(*samp_neighs) for samp_neighs in rel_sample_neighs]
#         unique_nodes_list = list(set.union(*multi_unique_nodes_list))
#         unique_nodes = {n:i for i,n in enumerate(unique_nodes_list)}
#         mask = torch.zeros(len(rel_idxs), len(nodes), len(unique_nodes)) # no matter what relationship between them
#         column_indices = [unique_nodes[n] for samp_neighs in rel_sample_neighs for samp_neigh in samp_neighs for n in samp_neigh]   
#         row_indices = [i for r,samp_neighs in enumerate(rel_sample_neighs) for i in range(len(samp_neighs)) for j in range(len(samp_neighs[i]))]
# #         rel_indices = [r for r,samp_neighs in enumerate(multi_samp_neighs) for i in range(len(samp_neighs)) for j in range(len(samp_neighs[i]))]
#         mask[rel_indices, row_indices, column_indices] = 1
        # no stack
#         mask = mask.reshape((-1, len(unique_nodes))) # for stack adjcent matrixs for different relations
#         if self.cuda:
#             mask = mask.cuda()
        # todo: normalize
        # no stack
#         num_neigh = mask.sum(2, keepdim=True)
#         thre_zeros = torch.zeros_like(mask)
#         # num_neigh = mask.sum()/(mask.shape[0])
#         # mask = mask.div(num_neigh)
#         mask = torch.where(num_neigh != 0, mask.div(num_neigh), thre_zeros)
#         if self.cuda:
#             embed_matrix = self.features(torch.LongTensor(unique_nodes_list).cuda())
#         else:
#         print('self.features type', type(self.features))
#         embed_matrix = self.features(torch.LongTensor(unique_nodes_list))

#         print(mask.shape, embed_matrix.shape)
#         if self.second_cut:
#             num_rels = len(multi_samp_neighs)
#             num_rels, num_multi_cur_nodes, num_next_nodes = mask.shape
#             print('mask reshape,', num_rels, num_multi_cur_nodes//num_rels, num_next_nodes)
            # no stack
#             mask = mask.reshape((num_rels, num_multi_cur_nodes//num_rels, num_next_nodes))
#             num_multi_next_nodes, num_emb_dim = embed_matrix.shape
#             embed_matrix = embed_matrix.reshape((num_rels, num_multi_next_nodes//num_rels, num_emb_dim))
#             to_feats = torch.bmm(mask, embed_matrix)
#             to_feats = torch.zeros(num_rels, num_multi_cur_nodes//num_rels, num_emb_dim)
#             for i in range(num_rels):
                # print(i, "mask[i]", mask[i], "embed_matrix[i]", embed_matrix[i])
#                 to_feats[i] = mask[i].mm(embed_matrix[i])
#             print('second cut : to_feats shape', to_feats.shape)
#             import numpy as np
#             print(np.where(np.isnan(to_feats.detach().numpy())==False))
#             to_feats = to_feats.sum(0)
#             to_feats = to_feats.reshape((num_multi_cur_nodes, num_emb_dim))
#         else:
#             # broadcast
#             print('first cut: mask shape and embed_matrix shape', mask.shape, embed_matrix.shape)
#             to_feats = torch.matmul(mask, embed_matrix)
#             print('first cut: to_feats shape', to_feats.shape)
#         print('rel_idxs',rel_idxs, 'torch.from_numpy', torch.from_numpy(rel_idxs))
        return aggr_rel_feats, torch.from_numpy(rel_idxs)

In [139]:
class RGCN_GraphSAGE(nn.Module):
    def __init__(self, features, num_feats, h_dim, num_rel, adj_lists):
        super(RGCN_GraphSAGE, self).__init__()
        self.encoder = Encoder(features, num_feats, h_dim, num_rel, adj_lists)
        self.weight_relation = nn.Parameter(
            torch.FloatTensor(num_rel, h_dim))
        nn.init.xavier_uniform_(self.weight_relation)
        
    def forward(self, triplets):
#         print(triplets[:,0])
        sub = self.encoder.forward(triplets[:,0])
        obj = self.encoder.forward(triplets[:,2])
        rel = self.weight_relation[triplets[:,1]]
#         print('sub obj rel shape', sub.shape, obj.shape, rel.shape)
        return torch.sum(sub*obj*rel, dim=1, keepdim=True)
    
    def loss(self, triplets, labels):
        score = self.forward(triplets)
#         print('score.type {}, labels.type {}'.format(score.dtype, labels.dtype))
#         print('score.shape {}, labels.shape {}'.format(score.shape, labels.shape))
        return F.binary_cross_entropy_with_logits(score, labels)

In [2]:
def load_dblp():
    num_nodes = 23260-1
    num_feats = 300
    num_rels = 10
    feat_data = np.zeros((num_nodes, num_feats))
#     labels = np.empty((num_nodes,1), dtype=np.int64)
    node_map = {}
    label_map = {}

    # feature
    with open("dblp/sub.node.dat") as fp:
        for i,line in enumerate(fp):
            info = line.strip().split()
            feat_data[i,:] = list(map(float, info[-1].split(',')))
#             labels[i] = float(info[-2])
            name = ' '.join(info[:-2])
            node_map[name] = i
            # print(labels[i])
            # if not info[-1] in label_map:
                # label_map[info[-1]] = len(label_map)
            # labels[i] = label_map[info[-1]]

    # adjcent matrixs for each relation
    adj_lists = [defaultdict(set) for i in range(num_rels)]
    triplets = []
#     labels = []
    with open("dblp/sub.link.dat") as fp:
        for i,line in enumerate(fp):
            info = map(int, line.strip().split())
            sub, obj, rel, _ = info
            triplets.append([sub, rel, obj])
#             labels.append(1)
            adj_lists[rel][sub].add(obj)
            

    # for debug puepose, print the dims
    # print(feat_data.shape, labels.shape)
    # for i in range(num_rels):
    #     print(i, len(adj_lists[i]))
    #     if i != 1:
    #         continue
    #     for key in adj_lists[i]:
    #         print(key, len(adj_lists[i][key]))

    return feat_data, adj_lists, np.array(triplets)